In [2]:
%pip install pandas transformers peft accelerate sentencepiece datasets 

ERROR: datasets 2.21.0 has requirement requests>=2.32.2, but you'll have requests 2.22.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import torch
# from modelscope import snapshot_download, AutoModel, AutoTokenizer
# import os
# model_dir = snapshot_download('qwen/Qwen2-7B-Instruct', cache_dir='/root/autodl-tmp', revision='master')

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
from peft import PeftModel,LoraConfig, TaskType, get_peft_model
from datasets import Dataset
import torch

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
train_df = pd.read_csv('./train.csv', encoding='gb2312', encoding_errors='ignore')

In [3]:
test_df = pd.read_csv('./test.csv', encoding='gb2312', encoding_errors='ignore')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   数据编号    3500 non-null   object
 1   创作要求    2000 non-null   object
 2   评判维度    3500 non-null   object
 3   待评判内容   3475 non-null   object
 4   标注分值    3500 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 136.8+ KB


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3049 entries, 0 to 3048
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   数据编号    3049 non-null   object
 1   创作要求    2049 non-null   object
 2   评判维度    3049 non-null   object
 3   待评判内容   3025 non-null   object
dtypes: object(4)
memory usage: 95.4+ KB


In [6]:
train_df[:2]

,数据编号,创作要求,评判维度,待评判内容,标注分值
0,TR0001,NaN,流畅性,美国塔科马市（47°17'N，122°28'W，是北太平洋东岸的港口城市，位于，人口约21....,3
1,TR0002,请以“针对中国人民银行及其各级分支机构，以及各类银行机构（包括政策性银行、国有商业银行、中国...,规范性,中国人民银行上海总部，各省、自治区、直辖市及计划单列市分行；各政策性银行、国有商业银行，中国...,1


In [7]:
train_df[-2:]

,数据编号,创作要求,评判维度,待评判内容,标注分值
3498,CH0499,边际贡献率和变动成本率____。 选择正确选项作答 A、反方向变化 B、同方向变化 C、同...,选择题,正确选项为A 模型回复为根据题干中的信息，我们可以推断出 反比例变化,0
3499,CH0500,"$HO_2$的沸点比$H_2S$的高,可以从____角度来进行解释。 选择正确选项作答 A...",选择题,正确选项为C 模型回复为C:氢键,1


In [8]:
test_df[:2]

,数据编号,创作要求,评判维度,待评判内容
0,TE0001,请以“描述一个初冬早晨的校园场景，展现了学生们在学业压力与日常生活小插曲中的生动画面，特别是...,规范性,那是一个潮湿的初冬的早上，学校那栋白色的教学楼像是融化在白雾里，只看见透明的或是绿色的玻璃窗...
1,TE0002,NaN,流畅性,2024年春节临近，山西博物院推出多款文创产品和多场展览推出，前来参观的游客不少吸引。\n2...


In [9]:
test_df[-2:]

,数据编号,创作要求,评判维度,待评判内容
3047,CHT1048,1948年底到1949年初，中共中央确定了一个重要的外交方针，将承认问题由新政权的单向“被承...,选择题,正确选项为D 模型回复为The correct answer is “求同存异”
3048,CHT1049,新冠肺炎疫情防控期间，“天价口罩”凸显了相关企业违反的商业伦理类型是 选择正确选项作答 A...,选择题,正确选项为D 模型回复为D.操纵价格行为


In [10]:
res = []
for i in range(len(train_df)):
    llm_item = train_df.loc[i]
    if (llm_item['评判维度']=="选择题"):
        tmp =  {
        "instruction": "如果模型回复与正确选项一致，则输出1；如果不一致，则输出0。",
        "input": f"{llm_item['创作要求']}。{llm_item['待评判内容']}",
        "output": str(llm_item['标注分值']) 
        }
        res.append(tmp)

    elif (llm_item['评判维度']=="流畅性"):
        tmp =   {
        "instruction": """下面是一个模型完成的创作内容。按照流畅性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。流畅性评分标准如下：
        1分: 非常不流畅，不具备可读性，语法错误明显，难以理解，大量拼写错误和错别字，影响阅读，表达不清晰，难以捉摸要表达的意思，每百字平均错误数超过2.5个。
        2分: 具有可读性，但较不流畅，常见语法错误多，需花费一定时间理解，一些拼写错误和错别字，阅读中断，表达较为模糊，需用一些猜测才能明白含义，每百字平均错误数介于2至2.5个。
        3分：基本流畅，存在少量语法错误，但影响较小，稍有拼写错误，但不影响阅读，主要意思表达清楚，但部分地方表述不够准确，每百字平均错误数介于1至2个。
        4分：较流畅，语法错误稀少，易读性较高，几乎无拼写错误，阅读顺畅，表达清晰、准确，容易理解，每百字平均错误数介于0.5至1个。
        5分：非常流畅，语法、拼写完美，阅读体验优秀，表达精炼、准确、得体；文句优美，行文连贯，思维严密，每百字平均错误数在0至0.5个之间。""",
        "input": f" 模型创作:{llm_item['待评判内容']}",
        "output": str(llm_item['标注分值']) 
        }
        res.append(tmp)

    else:
        tmp ={
        "instruction": """下面是一个模型，根据创作要求，完成的创作内容。按照规范性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。规范性评分标准如下：
        1分: 创作内容离题，与提示语句要求不符，格式非常不规范，杂乱无章，句子结构混乱，缺乏逻辑，每千字平均错误数超过5个。
        2分: 创作内容与提示语句要求有一定契合但覆盖不全，格式较不规范：缺乏清晰的结构，但基本逻辑仍能找到，每千字平均错误数在4至5个之间。
        3分：创作内容与提示语句要求基本契合但覆盖不全，格式一般规范，结构基本顺畅，逻辑较清晰，每千字平均错误数在2至4个之间。
        4分：创作内容与提示语句要求基本契合且基本覆盖，格式较规范，结构清晰，逻辑条理分明，每千字平均错误数在1至2个之间。
        5分：创作内容与提示语句要求完美契合，格式非常规范：结构严谨，逻辑清晰，层次分明，每千字平均错误数在0至1个之间。""",
        "input": f"创作要求:{llm_item['创作要求']}。模型创作:{llm_item['待评判内容']}",
        "output": str(llm_item['标注分值']) 
        }
        res.append(tmp)          

In [11]:
len(res)

3500

In [12]:
import json
import numpy as np

def convert_to_builtin_type(obj):
    if isinstance(obj, np.int64):  # 如果对象是 int64 类型
        return int(obj)  # 转换为内置的 int 类型
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

with open('./train_glm4.json', mode='w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4, default=convert_to_builtin_type)

In [13]:
import torch
from modelscope import snapshot_download, AutoModel, AutoTokenizer
import os

model_dir = snapshot_download('ZhipuAI/glm-4-9b-chat', cache_dir='/group_share/glm-4-9b-chat', revision='master')

2024-09-10 22:56:36,984 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-09-10 22:56:36,989 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-09-10 22:56:42,663 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-09-10 22:56:43,629 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 f6e031fdbad3477fb3f2c28e57d6c348 and a total number of 976 components indexed
2024-09-10 22:56:44,458 - modelscope - WARNING - Authentication has expired, please re-login if you need to access private models or datasets.
Downloading: 100%|██████████| 1.40k/1.40k [00:00<00:00, 12.7MB/s]
Downloading: 100%|██████████| 36.0/36.0 [00:00<00:00, 353kB/s]
Downloading: 100%|██████████| 2.21k/2.21k [00:00<00:00, 21.8MB/s]
Downloading: 100%|██████████| 207/207 [00:00<00:00, 1.90MB/s]
Downloading: 100%|██████████| 6.34k/6.34k [00:00<00:00, 43.6MB/s]
Downloading: 100%|██

In [10]:
def process_func(example):
    MAX_LENGTH = 2048
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer((f"[gMASK]<sop><|system|>\n您将担任评审的专家角色。<|user|>\n"
                            f"{example['instruction']+example['input']}<|assistant|>\n"
                            ).strip(), 
                            add_special_tokens=False)
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [11]:
df = pd.read_json('./train_glm4.json')
ds = Dataset.from_pandas(df)

In [12]:
ds[:2]

{'instruction': ['下面是一个模型完成的创作内容。按照流畅性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。流畅性评分标准如下：\n        1分: 非常不流畅，不具备可读性，语法错误明显，难以理解，大量拼写错误和错别字，影响阅读，表达不清晰，难以捉摸要表达的意思，每百字平均错误数超过2.5个。\n        2分: 具有可读性，但较不流畅，常见语法错误多，需花费一定时间理解，一些拼写错误和错别字，阅读中断，表达较为模糊，需用一些猜测才能明白含义，每百字平均错误数介于2至2.5个。\n        3分：基本流畅，存在少量语法错误，但影响较小，稍有拼写错误，但不影响阅读，主要意思表达清楚，但部分地方表述不够准确，每百字平均错误数介于1至2个。\n        4分：较流畅，语法错误稀少，易读性较高，几乎无拼写错误，阅读顺畅，表达清晰、准确，容易理解，每百字平均错误数介于0.5至1个。\n        5分：非常流畅，语法、拼写完美，阅读体验优秀，表达精炼、准确、得体；文句优美，行文连贯，思维严密，每百字平均错误数在0至0.5个之间。',
  '下面是一个模型，根据创作要求，完成的创作内容。按照规范性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。规范性评分标准如下：\n        1分: 创作内容离题，与提示语句要求不符，格式非常不规范，杂乱无章，句子结构混乱，缺乏逻辑，每千字平均错误数超过5个。\n        2分: 创作内容与提示语句要求有一定契合但覆盖不全，格式较不规范：缺乏清晰的结构，但基本逻辑仍能找到，每千字平均错误数在4至5个之间。\n        3分：创作内容与提示语句要求基本契合但覆盖不全，格式一般规范，结构基本顺畅，逻辑较清晰，每千字平均错误数在2至4个之间。\n        4分：创作内容与提示语句要求基本契合且基本覆盖，格式较规范，结构清晰，逻辑条理分明，每千字平均错误数在1至2个之间。\n        5分：创作内容与提示语句要求完美契合，格式非常规范：结构严谨，逻辑清晰，层次分明，每千字平均错误数在0至1个之间。'],
 'input': [" 模型创作:美国塔科马市（47°17'N，122°28'W，是北太平洋东岸的港口城市，位于，人口约21.9万（2020年

In [14]:
%pip install tiktoken

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/77/ba/e4149aa0724a5268ab10014a028668c6ef8834862f955ef0c6758240f6d1/tiktoken-0.7.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/3d/09/d82fe4a34c5f0585f9ea1df090e2a71eb9bb1e469723053e1ee9f57c16f3/charset_normalizer-3.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (141 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.22.0
    Not uninstalling requests at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'requests'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.


In [13]:
model = AutoModelForCausalLM.from_pretrained('/group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat', device_map="auto",torch_dtype=torch.bfloat16,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('/group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat', use_fast=True,trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 10/10 [00:05<00:00,  1.75it/s]


In [14]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 3500/3500 [00:05<00:00, 604.08 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3500
})

In [15]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))
print(tokenized_id[0]['input_ids'])
print(tokenizer.decode([151331, 151333, 151335]))
print(tokenizer.encode('[gMASK]<sop><|system|>', add_special_tokens=False))

[gMASK] <sop> <|system|> 
您将担任评审的专家角色。 <|user|> 
下面是一个模型完成的创作内容。按照流畅性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。流畅性评分标准如下：
        1分: 非常不流畅，不具备可读性，语法错误明显，难以理解，大量拼写错误和错别字，影响阅读，表达不清晰，难以捉摸要表达的意思，每百字平均错误数超过2.5个。
        2分: 具有可读性，但较不流畅，常见语法错误多，需花费一定时间理解，一些拼写错误和错别字，阅读中断，表达较为模糊，需用一些猜测才能明白含义，每百字平均错误数介于2至2.5个。
        3分：基本流畅，存在少量语法错误，但影响较小，稍有拼写错误，但不影响阅读，主要意思表达清楚，但部分地方表述不够准确，每百字平均错误数介于1至2个。
        4分：较流畅，语法错误稀少，易读性较高，几乎无拼写错误，阅读顺畅，表达清晰、准确，容易理解，每百字平均错误数介于0.5至1个。
        5分：非常流畅，语法、拼写完美，阅读体验优秀，表达精炼、准确、得体；文句优美，行文连贯，思维严密，每百字平均错误数在0至0.5个之间。 模型创作:美国塔科马市（47°17'N，122°28'W，是北太平洋东岸的港口城市，位于，人口约21.9万（2020年）。该市一位名叫约克的年轻人，非常喜爱中国文化的，工作之余经常前往图书馆读书，或漫步公园游憩。1873年，横贯美洲大陆的北太平洋铁路建成。这条铁路成了分处铁路两端的约克曾祖父母缔结姻缘的纽带。他们初识时，塔科马位于铁路西端，千余人仅居住于此，到1889年人口达3.6万。图12示意塔科马市内部空间结构。 <|assistant|> 3 <|endoftext|>
[151331, 151333, 151335, 198, 99526, 98493, 101638, 105724, 98314, 100153, 101053, 1773, 151336, 198, 101534, 99950, 100484, 109943, 100406, 99098, 1773, 99928, 108548, 98394, 104951, 99285, 98602, 100484, 100406, 98646, 98363, 7, 9855

In [16]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"])))

'3 <|endoftext|>'

In [17]:
model

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(151552, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-39): 40 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): SdpaAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in

In [18]:
model.enable_input_require_grads()

In [19]:
model.dtype

torch.bfloat16

In [20]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],  # 现存问题只微调部分演示即可
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value', 'dense', 'dense_4h_to_h', 'dense_h_to_4h'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [21]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value', 'dense', 'dense_4h_to_h', 'dense_h_to_4h'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [ ]:
model.print_trainable_parameters()

In [ ]:
args = TrainingArguments(
    output_dir="/group_share/glm4_9B_chat_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=50,
    num_train_epochs=5,
    save_steps=500,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [40]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [41]:
trainer.train() #不小心中断了

[2024-09-10 23:23:18,009] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/.conda/envs/xtuner0.1.17/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.1
 [WARNING]  using untested triton version (2.1.0), only 1.0.0 is known to be compatible


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
50,1.583500
100,0.626800
150,0.569200
200,0.544800
250,0.488000
300,0.441200
350,0.442500
400,0.415200
450,0.377900
500,0.317400


/root/.conda/envs/xtuner0.1.17/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat - will assume that the vocabulary was not modified.
  warnings.warn(
/root/.conda/envs/xtuner0.1.17/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=1090, training_loss=0.34173109367353105, metrics={'train_runtime': 18113.4973, 'train_samples_per_second': 0.966, 'train_steps_per_second': 0.06, 'total_flos': 1.0802587377095148e+18, 'train_loss': 0.34173109367353105, 'epoch': 4.982857142857143})

In [33]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer
 
 
def apply_lora(model_name_or_path, output_path, lora_path):
    print(f"Loading the base model from {model_name_or_path}")
    base = AutoModelForCausalLM.from_pretrained(
        model_name_or_path, torch_dtype=torch.float16, low_cpu_mem_usage=True
    )
    base_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
 
    print(f"Loading the LoRA adapter from {lora_path}")
 
    lora_model = PeftModel.from_pretrained(
        base,
        lora_path,
        torch_dtype=torch.bfloat16,
    )
 
    print("Applying the LoRA")
    model = lora_model.merge_and_unload()
 
    print(f"Saving the target model to {output_path}")
    model.save_pretrained(output_path)
    base_tokenizer.save_pretrained(output_path)

In [34]:
model_name_or_path = "/group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat"
output_path = "/group_share/glm4_9B_chat_instruct_review"
lora_path = "/group_share/glm4_9B_chat_lora/checkpoint-1000"

In [35]:
apply_lora(model_name_or_path, output_path, lora_path)

Loading the base model from /group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat


The repository for /group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for /group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Loading checkpoint shards: 100%|██████████| 10/10 [00:06<00:00,  1.59it/s]


The repository for /group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//group_share/glm-4-9b-chat/ZhipuAI/glm-4-9b-chat.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Loading the LoRA adapter from /group_share/glm4_9B_chat_lora/checkpoint-1000
Applying the LoRA
Saving the target model to /group_share/glm4_9B_chat_instruct_review


In [46]:
%pip install modelscope

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/58/bd/963488515859b641a2aa525ed20282e914dd4d3dc9471002f11b94e4a016/modelscope-1.18.0-py3-none-any.whl (5.7 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/ca/1c/89ffc63a9605b583d5df2be791a27bc1a42b7c32bab68d3c8f2f73a98cd4/urllib3-2.2.2-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.8
    Not uninstalling urllib3 at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'urllib3'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.


In [49]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 9 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 https://mirrors.aliyun.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 2s (1487 kB/s)  
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package git-lfs.
(Reading database ... 27010 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
Unpacking git-lfs (2.9.2-1) ...
Setting up git-lfs (2.9.2-1) ...


In [ ]:
from modelscope.hub.api import HubApi

YOUR_ACCESS_TOKEN = 'e65f0ae8-fd82-4626-a011-c1001e776ccd'

api = HubApi()
api.login(YOUR_ACCESS_TOKEN)
api.push_model(
    model_id="NumberJys/glm4_9B_chat_review_1000", 
    model_dir="/group_share/glm4_9B_chat_instruct_review" # 本地模型目录，要求目录中必须包含configuration.json
)

2024-09-11 10:04:39,215 - modelscope - INFO - [master 06569cf] 'upload model'
 16 files changed, 2141 insertions(+), 52 deletions(-)
 delete mode 100644 README.md
 create mode 100644 added_tokens.json
 create mode 100644 config.json
 create mode 100644 configuration_chatglm.py
 create mode 100644 generation_config.json
 create mode 100644 model-00001-of-00004.safetensors
 create mode 100644 model-00002-of-00004.safetensors
 create mode 100644 model-00003-of-00004.safetensors
 create mode 100644 model-00004-of-00004.safetensors
 create mode 100644 model.safetensors.index.json
 create mode 100644 modeling_chatglm.py
 create mode 100644 special_tokens_map.json
 create mode 100644 tokenization_chatglm.py
 create mode 100644 tokenizer.model
 create mode 100644 tokenizer_config.json



In [36]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

model_path = '/group_share/glm4_9B_chat_instruct_review'
# lora_path = '/group_share/glm4_9B_chat_lora/checkpoint-1000'

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path,use_fast=True,trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto",torch_dtype=torch.bfloat16,trust_remote_code=True)

# # 加载lora权重
# model = PeftModel.from_pretrained(model, model_id=lora_path)

The repository for /group_share/glm4_9B_chat_instruct_review contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//group_share/glm4_9B_chat_instruct_review.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for /group_share/glm4_9B_chat_instruct_review contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//group_share/glm4_9B_chat_instruct_review.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y
The repository for /group_share/glm4_9B_chat_instruct_review contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//group_share/glm4_9B_chat_instruct_review.
You can avoid this prompt in future b

Loading checkpoint shards: 100%|██████████| 4/4 [01:33<00:00, 23.37s/it]


In [37]:
prompt = "如果模型回复与正确选项一致，则输出1；如果不一致，则输出0。火灾自动报警系统可燃气体探测器，撤去可燃气体，探测器应在____内恢复到正常监视状态。 选择正确选项作答  A、60s B、80s C、100s D、120s。正确选项为A 模型回复为\"\"\"60s *\"\"\""
inputs = tokenizer.apply_chat_template([{"role": "system", "content": "您将担任评审的专家角色"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2048, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

1


In [38]:
def predict1(text):
    
    prompt=f"""以下是一个大模型完成的选择题。如果大模型的回答与参考答案一致，输出1；如果不一致，输出0。
            -->{text} """
    inputs = tokenizer.apply_chat_template([{"role": "system", "content": "您将担任评审的专家角色"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


    gen_kwargs = {"max_length": 2048, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [39]:
def predict2(text):
    
    prompt=f"""下面是一个模型完成的创作内容。按照流畅性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。流畅性评分标准如下： 
            1分: 非常不流畅，不具备可读性，语法错误明显，难以理解，大量拼写错误和错别字，影响阅读，表达不清晰，难以捉摸要表达的意思，每百字平均错误数超过2.5个。 
            2分: 具有可读性，但较不流畅，常见语法错误多，需花费一定时间理解，一些拼写错误和错别字，阅读中断，表达较为模糊，需用一些猜测才能明白含义，每百字平均错误数介于2至2.5个。 
            3分：基本流畅，存在少量语法错误，但影响较小，稍有拼写错误，但不影响阅读，主要意思表达清楚，但部分地方表述不够准确，每百字平均错误数介于1至2个。 
            4分：较流畅，语法错误稀少，易读性较高，几乎无拼写错误，阅读顺畅，表达清晰、准确，容易理解，每百字平均错误数介于0.5至1个。 
            5分：非常流畅，语法、拼写完美，阅读体验优秀，表达精炼、准确、得体；文句优美，行文连贯，思维严密，每百字平均错误数在0至0.5个之间。
            -->{text} """
    inputs = tokenizer.apply_chat_template([{"role": "system", "content": "您将担任评审的专家角色"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


    gen_kwargs = {"max_length": 2048, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [40]:
def predict3(text):
    
    prompt=f"""下面是一个模型，根据创作要求，完成的创作内容。按照规范性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。规范性评分标准如下： 
            1分: 创作内容离题，与提示语句要求不符，格式非常不规范，杂乱无章，句子结构混乱，缺乏逻辑，每千字平均错误数超过5个。 
            2分: 创作内容与提示语句要求有一定契合但覆盖不全，格式较不规范：缺乏清晰的结构，但基本逻辑仍能找到，每千字平均错误数在4至5个之间。 
            3分：创作内容与提示语句要求基本契合但覆盖不全，格式一般规范，结构基本顺畅，逻辑较清晰，每千字平均错误数在2至4个之间。 
            4分：创作内容与提示语句要求基本契合且基本覆盖，格式较规范，结构清晰，逻辑条理分明，每千字平均错误数在1至2个之间。 
            5分：创作内容与提示语句要求完美契合，格式非常规范：结构严谨，逻辑清晰，层次分明，每千字平均错误数在0至1个之间。
            -->{text} """
    inputs = tokenizer.apply_chat_template([{"role": "system", "content": "您将担任评审的专家角色"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


    gen_kwargs = {"max_length": 2048, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [41]:
from tqdm import tqdm
label = []
for i in tqdm(range(len(test_df))):
    test_item = test_df.loc[i]
    if (test_item['评判维度']=="选择题"):
        test_input = f"{test_item['创作要求']}。{test_item['待评判内容']}"
        label.append(int(predict1(test_input)))
    elif (test_item['评判维度']=="流畅性"):
        test_input = f"模型创作:{test_item['待评判内容']}"
        label.append(int(predict2(test_input)))
    else:
        test_input = f"创作要求:{test_item['创作要求']}。模型创作:{test_item['待评判内容']}"
        label.append(int(predict3(test_input)))

100%|██████████| 3049/3049 [08:23<00:00,  6.06it/s]


In [42]:
test_df['预测分数'] = label

In [43]:
submit = test_df[['数据编号', '评判维度', '预测分数']]

In [44]:
submit.to_csv('glm4_9b_1000.csv', index=False)

In [ ]:
from tqdm import tqdm
label = []
for i in tqdm(range(len(test_df))):
    test_item = test_df.loc[i]
    if (test_item['评判维度']=="选择题"):
        test_input = f"{test_item['创作要求']}。{test_item['待评判内容']}"
        label.append(int(predict1(test_input)))
    elif (test_item['评判维度']=="流畅性"):
        test_input = f"模型创作:{test_item['待评判内容']}"
        label.append(int(predict2(test_input)))
    else:
        test_input = f"创作要求:{test_item['创作要求']}。模型创作:{test_item['待评判内容']}"
        label.append(int(predict3(test_input)))y

In [1]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer
 
 
def apply_lora(model_name_or_path, output_path, lora_path):
    print(f"Loading the base model from {model_name_or_path}")
    base = AutoModelForCausalLM.from_pretrained(
        model_name_or_path, torch_dtype=torch.float16, low_cpu_mem_usage=True
    )
    base_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
 
    print(f"Loading the LoRA adapter from {lora_path}")
 
    lora_model = PeftModel.from_pretrained(
        base,
        lora_path,
        torch_dtype=torch.bfloat16,
    )
 
    print("Applying the LoRA")
    model = lora_model.merge_and_unload()
 
    print(f"Saving the target model to {output_path}")
    model.save_pretrained(output_path)
    base_tokenizer.save_pretrained(output_path)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name_or_path = "/root/share/new_models/qwen/Qwen2-7B-Instruct"
output_path = "/group_share/Qwen2_7B_instruct_review"
lora_path = "/group_share/Qwen2_7B_instruct_lora/checkpoint-500"

In [3]:
apply_lora(model_name_or_path, output_path, lora_path)

Loading the base model from /root/share/new_models/qwen/Qwen2-7B-Instruct


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.01s/it]


Loading the LoRA adapter from /group_share/Qwen2_7B_instruct_lora/checkpoint-500
Applying the LoRA
Saving the target model to /group_share/Qwen2_7B_instruct_review


In [5]:
%pip install modelscope

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/58/bd/963488515859b641a2aa525ed20282e914dd4d3dc9471002f11b94e4a016/modelscope-1.18.0-py3-none-any.whl (5.7 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/ca/1c/89ffc63a9605b583d5df2be791a27bc1a42b7c32bab68d3c8f2f73a98cd4/urllib3-2.2.2-py3-none-any.whl (121 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/3d/09/d82fe4a34c5f0585f9ea1df090e2a71eb9bb1e469723053e1ee9f57c16f3/charset_normalizer-3.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (141 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.8
    Not uninstalling urllib3 at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'urllib3'. No files were found to 

In [10]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 9 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 https://mirrors.aliyun.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 4s (766 kB/s)   33m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package git-lfs.
(Reading database ... 27010 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking git-lfs (2.9.2-1) ...
7Progress: [ 40%] [#######################...................................] 8Setting up git-lfs 

In [ ]:
from modelscope.hub.api import HubApi

YOUR_ACCESS_TOKEN = 'e65f0ae8-fd82-4626-a011-c1001e776ccd'

api = HubApi()
api.login(YOUR_ACCESS_TOKEN)
api.push_model(
    model_id="NumberJys/Qwen2_7B_instruct_review_500", 
    model_dir="/group_share/Qwen2_7B_instruct_review" # 本地模型目录，要求目录中必须包含configuration.json
)